In [2]:
import numpy as np
import statsmodels.api as sm
import linearmodels.iv.model as lm
from scipy import stats
import pandas as pd

In [3]:
df = pd.read_excel('wooldridge_wage.xlsx')

In [4]:
df

,wage,hours,IQ,KWW,educ,exper,tenure,age,married,black,south,urban,sibs,brthord,meduc,feduc,lwage
0,769,40,93,35,12,11,2,31,1,0,0,1,1,2,8,8,6.645091
1,825,40,108,46,14,11,9,33,1,0,0,1,1,2,14,14,6.715384
2,650,40,96,32,12,13,7,32,1,0,0,1,4,3,12,12,6.476973
3,562,40,74,27,11,14,5,34,1,0,0,1,10,6,6,11,6.331502
4,600,40,91,24,10,13,0,30,0,0,0,1,1,2,8,8,6.396930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
658,1442,40,113,45,16,8,10,35,1,0,1,1,2,2,8,8,7.273787
659,645,45,93,39,12,11,3,35,1,0,1,0,7,7,7,8,6.469250
660,477,45,100,33,12,9,3,31,1,0,1,0,3,3,7,7,6.167517
661,664,60,82,30,16,10,9,34,1,1,1,1,3,4,16,16,6.498282


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 663 entries, 0 to 662
Data columns (total 17 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   wage     663 non-null    int64  
 1   hours    663 non-null    int64  
 2   IQ       663 non-null    int64  
 3   KWW      663 non-null    int64  
 4   educ     663 non-null    int64  
 5   exper    663 non-null    int64  
 6   tenure   663 non-null    int64  
 7   age      663 non-null    int64  
 8   married  663 non-null    int64  
 9   black    663 non-null    int64  
 10  south    663 non-null    int64  
 11  urban    663 non-null    int64  
 12  sibs     663 non-null    int64  
 13  brthord  663 non-null    int64  
 14  meduc    663 non-null    int64  
 15  feduc    663 non-null    int64  
 16  lwage    663 non-null    float64
dtypes: float64(1), int64(16)
memory usage: 88.2 KB


In [6]:
df.describe()

,wage,hours,IQ,KWW,educ,exper,tenure,age,married,black,south,urban,sibs,brthord,meduc,feduc,lwage
count,663.000000,663.000000,663.000000,663.000000,663.000000,663.000000,663.000000,663.000000,663.000000,663.000000,663.000000,663.000000,663.000000,663.000000,663.000000,663.000000,663.000000
mean,988.475113,44.061840,102.481146,36.194570,13.680241,11.396682,7.217195,32.983409,0.900452,0.081448,0.322775,0.719457,2.846154,2.177979,10.828054,10.273002,6.814297
std,406.511512,7.159856,14.686117,7.529188,2.231406,4.258397,5.055690,3.062989,0.299622,0.273728,0.467891,0.449604,2.240896,1.487612,2.823188,3.288170,0.412206
min,115.000000,25.000000,54.000000,13.000000,9.000000,1.000000,0.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,4.744932
25%,699.000000,40.000000,94.000000,32.000000,12.000000,8.000000,3.000000,30.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,9.000000,8.000000,6.549650
50%,937.000000,40.000000,104.000000,37.000000,13.000000,11.000000,7.000000,33.000000,1.000000,0.000000,0.000000,1.000000,2.000000,2.000000,12.000000,11.000000,6.842683
75%,1200.000000,48.000000,113.000000,41.000000,16.000000,15.000000,11.000000,36.000000,1.000000,0.000000,1.000000,1.000000,4.000000,3.000000,12.000000,12.000000,7.090077
max,3078.000000,80.000000,145.000000,56.000000,18.000000,22.000000,22.000000,38.000000,1.000000,1.000000,1.000000,1.000000,14.000000,10.000000,18.000000,18.000000,8.032035


In [7]:
df['exper']

0      11
1      11
2      13
3      14
4      13
       ..
658     8
659    11
660     9
661    10
662    10
Name: exper, Length: 663, dtype: int64

In [19]:
np.random.seed(42)
# Generate data
n = 10000
Y=df['lwage']   #dependent variable
X1 = df['exper']  # Exogenous variable
X2 = df['educ']   #endogenous variable 
instrument = df[['meduc','feduc']]  # Instrument for X2
epsilon = np.random.randn(n)  # Error term

In [20]:
X1.shape,X2.shape

((663,), (663,))

In [21]:
Y.shape

(663,)

In [22]:
# Original Regression
X_orig = sm.add_constant(np.column_stack((X1, X2)))  #Add a column of ones to an array (in this case adding columns X1 and X2)
model_original = sm.OLS(Y, X_orig).fit()   #estimating regression parameters using OLS using endog and exog varaibles 
orig_coef = model_original.params[2]
model_original.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  lwage   R-squared:                       0.145
Model:                            OLS   Adj. R-squared:                  0.143
Method:                 Least Squares   F-statistic:                     56.11
Date:                Fri, 01 Sep 2023   Prob (F-statistic):           3.12e-23
Time:                        22:26:19   Log-Likelihood:                -300.63
No. Observations:                 663   AIC:                             607.3
Df Residuals:                     660   BIC:                             620.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.4885      0.129     42.518      0.000       5.235       5.742
x1             0.0221      0.004      5.652      0.000       0.014       0.030
x2             0.0785      0.007     10.546      0.000       0.064       0.093
==============================================================================
Omnibus:                       18.775   Durbin-Watson:                   1.846
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               26.410
Skew:                          -0.266   Prob(JB):                     1.84e-06
Kurtosis:                       3.820   Cond. No.                         157.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [23]:
# Or you can use linearmodels package to do 2SLS and generate test result as well
mlr2 = lm.IV2SLS(dependent=Y, exog=X1, endog=X2, instruments=instrument).fit(cov_type="homoskedastic", debiased=True) #Estimation of IV models using two-stage least squares
print(mlr2.wu_hausman())   #using hausman test to test for endogeneity

"""Test statistic is difference between sum of squared OLS and sum of
squared IV residuals where each set of residuals has been projected
onto the set of instruments in the IV model"""

Wu-Hausman test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 40.7166
P-value: 0.0000
Distributed: F(1,660)


'Test statistic is difference between sum of squared OLS and sum of\nsquared IV residuals where each set of residuals has been projected\nonto the set of instruments in the IV model'

If the p-value from the chi-squared test is small (typically below a chosen significance level like 0.05), it suggests that the null hypothesis of no systematic difference between the OLS and IV coefficient estimates should be rejected. In this case, there is evidence of endogeneity, indicating that the OLS estimates are biased and inconsistent.

In [24]:
# First stage
X = sm.add_constant(np.column_stack((X1, instrument))) #Add a column of ones to an array (in this case adding columns X1 and IV)
model_first_stage = sm.OLS(X2, X).fit()
X2_hat = model_first_stage.predict(X)  # Predicted values of X2
model_first_stage.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   educ   R-squared:                       0.325
Model:                            OLS   Adj. R-squared:                  0.322
Method:                 Least Squares   F-statistic:                     105.7
Date:                Fri, 01 Sep 2023   Prob (F-statistic):           7.36e-56
Time:                        22:27:00   Log-Likelihood:                -1342.2
No. Observations:                 663   AIC:                             2692.
Df Residuals:                     659   BIC:                             2710.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         12.7711      0.393     32.503      0.000      12.000      13.543
x1            -0.1903      0.017    -10.969      0.000      -0.224      -0.156
x2             0.1325      0.031      4.276      0.000       0.072       0.193
x3             0.1600      0.027      5.898      0.000       0.107       0.213
==============================================================================
Omnibus:                       19.242   Durbin-Watson:                   1.996
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               19.020
Skew:                           0.380   Prob(JB):                     7.41e-05
Kurtosis:                       2.667   Cond. No.                         105.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [25]:
# Second stage
X_main = sm.add_constant(np.column_stack((X1, X2_hat)))  #you obtain the predicted values of the endogenous variable based on the instrumental variables.
model_second_stage = sm.OLS(Y, X_main).fit()
second_stage_coef = model_second_stage.params[2]
model_second_stage.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  lwage   R-squared:                       0.083
Model:                            OLS   Adj. R-squared:                  0.080
Method:                 Least Squares   F-statistic:                     29.78
Date:                Fri, 01 Sep 2023   Prob (F-statistic):           4.13e-13
Time:                        22:27:05   Log-Likelihood:                -324.04
No. Observations:                 663   AIC:                             654.1
Df Residuals:                     660   BIC:                             667.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.2983      0.326     13.171      0.000       3.657       4.939
x1             0.0392      0.006      6.651      0.000       0.028       0.051
x2             0.1512      0.020      7.657      0.000       0.112       0.190
==============================================================================
Omnibus:                       28.130   Durbin-Watson:                   1.856
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               41.146
Skew:                          -0.362   Prob(JB):                     1.16e-09
Kurtosis:                       3.982   Cond. No.                         382.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""